In [ ]:
import pandas as pd
Tenders = pd.read_excel("C:/Users/Mitch/git/UWACapstoneG2/data/UpdatedTenders.xlsx")

In [ ]:
CleanTenders = Tenders[["Reference Number", "TenderLink"]].dropna(subset=["TenderLink"]).drop_duplicates()
TenderDict = dict(zip(CleanTenders["Reference Number"], CleanTenders["TenderLink"]))

In [ ]:
# Extracts tender zip files from links
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# buttons 
locators = {
    "Download Now": By.LINK_TEXT,
    "Download for Information Only": By.XPATH,
    "Download Documents": By.XPATH
}

custom_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"

def click_button(driver, wait, button_text):
    
    locator = locators[button_text]
    if locator == "xpath":
        button_text = f"//input[@value='{button_text}']"
    
    try:
        button = wait.until(EC.element_to_be_clickable((locator, button_text)))
        button.click()
    except Exception as e:
        print(f"Error while clicking button '{button_text}': {e}")

def get_tenders(link, path):
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument(f"user-agent={custom_user_agent}")
    prefs = {"download.default_directory": path}
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    driver.get(link)

    # wait for button to be clickable + delay requests
    wait = WebDriverWait(driver, 10)
    
    buttons = ["Download Now", "Download for Information Only", "Download Documents"]
    for button in buttons:
        click_button(driver, wait, button)

    # enough time to download files before quitting
    time.sleep(5)
    driver.quit()

In [ ]:
FilterdTenders = {key: value for key, value in TenderDict.items() if "qas" not in value}

In [ ]:
# get num workers, how many files to download at the same time
num_threads = 10 
path = "C:\\Users\\Mitch\\Capstone\\Tenders"
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    for ref, link in FilterdTenders.items():
        executor.submit(get_tenders, link, path)